In [2]:
import numpy as np
import pandas as pd
import os
from datetime import datetime

from decimal import Decimal

import re

import warnings

import duckdb

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

In [3]:
df_duffield = pd.read_excel('CleanedDataFiles/DuffieldHall_2023.xlsx')
df_duffield.head()

,name,slottime,Chilled Water,Electric,Steam
0,DuffieldHall,2023-01-01 05:00:00,146.103,695.9,1942.7230
1,DuffieldHall,2023-01-01 05:15:00,147.651,636.8,604.3378
2,DuffieldHall,2023-01-01 05:30:00,145.867,657.8,0.0000
3,DuffieldHall,2023-01-01 05:45:00,145.797,666.7,0.0000
4,DuffieldHall,2023-01-01 06:00:00,145.125,662.4,0.0000


In [3]:
def hex_color_map(df, measure):
    # Normalize the measure values from 0 to 1
    normalized_measure = (df[measure] - df[measure].min()) / (df[measure].max() - df[measure].min())

    # Define colors for the custom colormap (dark dark blue to dark yellow)
    colors = [(0, 0, 0.5), (0.5, 0.5, 0)]  # RGB for dark dark blue and dark yellow
    cmap_name = 'custom_colormap'
    n_bins = 100  # Increase this for smoother color transitions

    # Create a custom colormap
    cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)

    # Map normalized values to the colormap
    colors = cm(normalized_measure)

    # Convert colors to hex format for JavaScript
    hex_colors = ['#' + ''.join([format(int(c*255), '02x') for c in color[:3]]) for color in colors]

    return hex_colors

In [4]:
def aggregate_month_helper(df, measure):
    
    df2 = df.copy()

    year_temp = []
    month_temp = []

    for i in range(len(df2)):
        year_temp.append(df2['slottime'][i].year)
        month_temp.append(df2['slottime'][i].month)

    df2['year'] = year_temp
    df2['month'] = month_temp

    df2 = df2[df2['year'] == 2023]
    df2.rename(columns = {'Chilled Water':'chilled_water'}, inplace = True) 

    df3 = duckdb.sql('SELECT name, month, AVG(chilled_water) AS chilled_water, AVG(Electric) as electric, \
                 AVG(Steam) AS steam FROM df2 GROUP BY name, month ORDER by month').df()
        
    return hex_color_map(df3, measure)

def aggregate_day_helper(df, which_month, measure):
    
    df2 = df.copy()

    year_temp = []
    month_temp = []
    day_temp = []

    for i in range(len(df2)):
        year_temp.append(df2['slottime'][i].year)
        month_temp.append(df2['slottime'][i].month)
        day_temp.append(df2['slottime'][i].day)

    df2['year'] = year_temp
    df2['month'] = month_temp
    df2['day'] = day_temp

    df2 = df2[df2['year'] == 2023]
    df2 = df2[df2['month'] == which_month]
    df2.rename(columns = {'Chilled Water':'chilled_water'}, inplace = True)

    df3 = duckdb.sql('SELECT name, day, AVG(chilled_water) AS chilled_water, AVG(Electric) as electric, \
                 AVG(Steam) AS steam FROM df2 GROUP BY name, day ORDER BY day').df()
        
    return hex_color_map(df3, measure)

def aggregate_hour_helper(df, which_month, which_day, measure):
    
    df2 = df.copy()

    year_temp = []
    month_temp = []
    day_temp = []
    hour_temp = []

    for i in range(len(df2)):
        year_temp.append(df2['slottime'][i].year)
        month_temp.append(df2['slottime'][i].month)
        day_temp.append(df2['slottime'][i].day)
        hour_temp.append(df2['slottime'][i].hour)

    df2['year'] = year_temp
    df2['month'] = month_temp
    df2['day'] = day_temp
    df2['hour'] = hour_temp

    df2 = df2[df2['year'] == 2023]
    df2 = df2[df2['month'] == which_month]
    df2 = df2[df2['day'] == which_day]
    df2.rename(columns = {'Chilled Water':'chilled_water'}, inplace = True)

    df3 = duckdb.sql('SELECT name, hour, AVG(chilled_water) AS chilled_water, AVG(Electric) as electric, \
                 AVG(Steam) AS steam FROM df2 GROUP BY name, hour ORDER BY hour').df()
        
    return hex_color_map(df3, measure)

In [5]:
def aggregate_colors(df, agg_by = 'month', which_month = None, which_day = None, measure = 'electric'):
    if agg_by == 'month':
        return aggregate_month_helper(df, measure)
    elif agg_by == 'day':
        if which_month == None:
            raise TypeError("aggregate_colors is missing 1 required positional argument: 'which_month'")
        return aggregate_day_helper(df, which_month, measure)
    elif agg_by == 'hour':
        if which_month == None: 
            raise TypeError("aggregate_colors is missing 1 required positional argument: 'which_month'")
        if which_day == None: 
            raise TypeError("aggregate_colors is missing 1 required positional argument: 'which_day'")
        return aggregate_hour_helper(df, which_month, which_day, measure)
    else:
        raise ValueError("Parameter 'agg_by' must be one of the following: month, day, hour.")

In [6]:
aggregate_colors(df_duffield, agg_by = 'hour', which_month = 3, which_day = 11)

NameError: name 'df_duffield' is not defined